### Query Enhancement – Query Expansion Techniques

In a RAG pipeline, the quality of the query sent to the retriever determines how good the retrieved context is — and therefore, how accurate the LLM’s final answer will be.

That’s where Query Expansion / Enhancement comes in.

#### 🎯 What is Query Enhancement?
Query enhancement refers to techniques used to improve or reformulate the user query to retrieve better, more relevant documents from the knowledge base.
It is especially useful when:

- The original query is short, ambiguous, or under-specified
- You want to broaden the scope to catch synonyms, related phrases, or spelling variants

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

In [3]:
# Step1: Load and split the dataset

loader = TextLoader("langchain_crewai_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 50)
chunks = splitter.split_documents(raw_docs)


In [4]:
### Step 2: vectore Store

embeding_model = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeding_model)


### step3: MMR Retriever 
retriever = vectorstore.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k":5}
)

e:\workspace\ultimate_rag_bootcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020E2BAB2120>, search_type='mmr', search_kwargs={'k': 5})

In [28]:
# Step 4: LLM and Prompt

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = init_chat_model("openai:gpt-4o-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020E70DC6F90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000020E70DC72F0>, root_client=<openai.OpenAI object at 0x0000020E70DC68D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000020E70DC70B0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [29]:
# Query expansion
query_expansion_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.

Original query: "{query}"

Expanded query:
""")

query_expansion_chain=query_expansion_prompt| llm | StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.\n\nOriginal query: "{query}"\n\nExpanded query:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020E70DC6F90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000020E70DC72F0>, root_client=<openai.OpenAI object at 0x0000020E70DC68D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000020E70DC70B0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [30]:
query_expansion_chain.invoke({"query":"Langchain memory"})

'"LangChain memory architecture, persistent memory storage, long-term memory in LangChain, memory management in LangChain, LangChain state management, memory optimization techniques in LangChain, LangChain context retention, algorithms for memory in LangChain, stateful interactions with LangChain, LangChain cognitive memory solutions, LangChain knowledge management, enhanced memory capabilities in LangChain applications, transformer memory mechanisms, LangChain use cases for memory implementation, comparison of memory systems in natural language processing with LangChain."'

In [31]:
## RAG answering prompt

answer_prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context: 
{context}

Question: {input}
""")

document_chain = create_stuff_documents_chain(llm=llm, prompt=answer_prompt)

In [33]:
### Step5: Full RAG pipeline with query expansion

rag_pipeline = (
    RunnableMap({
        "input": lambda x:x["input"],
        "context": lambda x: retriever.invoke(query_expansion_chain.invoke({"query":x["input"]}))
    })
    | document_chain
)

In [35]:
# Step 6: Run query
query = {"input": "What types of memory does LangChain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

Here's an expanded version of your query to improve document retrieval:

```
{
  'input': 'What types of memory architectures, storage mechanisms, and data retention systems does LangChain support? Specifically, I am looking for information on various memory types such as in-memory storage, persistent storage, short-term memory, long-term memory, and dynamic memory management within LangChain. Additionally, please include technical details about the APIs, connectors, and configurations related to memory implementation in LangChain. Are there any specific use cases, best practices, or examples of memory usage in LangChain applications?'
}
``` 

This expanded query includes synonyms and related terms that enhance specificity, targeting various aspects of memory and its implementation within LangChain.
✅ Answer:
 LangChain supports memory modules such as ConversationBufferMemory and ConversationSummaryMemory. These modules help the LLM maintain awareness of previous conversation turns or 

In [36]:
# Step 6: Run query
query = {"input": "CrewAI agents?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

Expanded query: 

"{'input': 'CrewAI agents, Crew AI personnel, Crew AI virtual assistants, Crew AI autonomous agents, Crew AI robotics, online crew management systems, AI-powered crew coordination, automated crew management solutions, artificial intelligence in crew operations, CrewAI technology, intelligent crew systems, digital crew agents, AI-driven crew optimization, efficiency in crew tasks, virtual crew collaboration tools, tools for enhancing crew performance, advancements in crew AI technology'}"
✅ Answer:
 CrewAI agents are autonomous agents that operate within a structured workflow framework. Each agent has a specific role—such as researcher, planner, or executor—and works semi-independently while collaborating with other agents in the crew. They are defined by a purpose, a goal, and a set of tools to use, ensuring that they stay on task and contribute to the overall objectives of the crew.
